In [30]:
trace = [i for i in range(0, 100)]

In [31]:
from Field import  *
from Polynomial import  *
exp_factor = 8
field = Field.main()
g = field.primitive_nth_root(128)
h = field.primitive_nth_root(128*exp_factor)

G = [g^i for i in range(len(trace))]
H = [h^i for i in range(128*exp_factor)]


In [32]:
trace = [FieldElement(i, Field.main()) for i in trace]

In [33]:
fx = Polynomial.interpolate_domain(G, trace)
fx

In [34]:
random_w = FieldElement(4, Field.main())
H_w = [h*random_w for h in H]
fx_exp_eval = fx.evaluate_domain(H_w)


In [35]:
from Merkle import *
print(len(fx_exp_eval))
fx_root =  MerkleTree(fx_exp_eval)
fx_root

1024


# Contrains
3 constrains
final and initial value must be the same
and intermidiate value must be the following formula:
$a_n = a_{n-1} + m$
where m is the slope of the line

In [36]:
p0_num = fx - Polynomial([FieldElement(0, Field.main())])
x = Polynomial([field.zero(), field.one()])
p0_dem = x - Polynomial([G[0]])
p0 = p0_num/p0_dem
p0

In [37]:
p1_num = fx - Polynomial([FieldElement(99, Field.main())])
x = Polynomial([field.zero(), field.one()])
p1_dem = x - Polynomial([G[99]])
p1 = p1_num/p1_dem
p1

In [38]:
p2_const = FieldElement(0, Field.main())
p2_num = fx.scale(g) - fx - Polynomial([FieldElement(1, Field.main())])
x = Polynomial([field.zero(), field.one()])
p2_dem = Polynomial([FieldElement(1, Field.main())])
for i in range(1, 99):
    p2_dem *= (x - Polynomial([G[i]]))
p2 =  p2_num/p2_dem
p2

In [39]:
cp = p0 + p1 + p2
cp_eval = cp.evaluate_domain(H_w)
mk = MerkleTree(cp_eval)

# FRI

In [40]:
from FRI import *

fri = Fri(H_w)

fri_polys, fri_domains, fri_layers, fri_merkles = fri.commit(cp, H_w, cp_eval, mk, None)

next_len fri layer: 512
next_len fri layer: 256
next_len fri layer: 128
next_len fri layer: 64
next_len fri layer: 32
next_len fri layer: 16
next_len fri layer: 8


In [41]:
#fri.decommit(1, fri_polys, fri_domains, fri_layers, fri_merkles)
[len(i) for i in fri_layers]

[1024, 512, 256, 128, 64, 32, 16, 8]

# Verify




In [42]:
mt_root = fx_root 

In [43]:
# 3 cp so 3 alphas
alpha0, alpha1,alpha2 = FieldElement(1, Field.main()), FieldElement(1, Field.main()), FieldElement(1, Field.main())

In [44]:
# CP merkle root
cp_root = mk

In [45]:
idx = 1

In [46]:
proof_f = fri.decommit_on_query(idx, fx_exp_eval, mt_root)

In [47]:
proof_cp = fri.decommit_on_fri(idx, fri_polys, fri_domains, fri_layers, fri_merkles)


In [48]:
# Basic check
assert len(proof_cp) % 2
v_last_one = proof_cp.pop()

In [49]:
last_one = fri_polys[-1].coefficients[0] # poly 0 shall be constant

v_last_one
assert v_last_one == last_one


In [50]:
from Merkle import verify_decommitment

In [51]:
# Check polynomial constraint
# First check evaluation of polynomial f is honest
v_f_eval = proof_f[0::2]
v_f_auth = proof_f[1::2]
assert len(v_f_eval) == len(v_f_auth)
for i in range(len(v_f_eval)):
    assert verify_decommitment(idx + 8 * i, v_f_eval[i], v_f_auth[i], mt_root.root ), f'in iter {i}, go wrong'
# Then check whether they could satify the recursive condition
v_cp0 = proof_cp[:2]
length = len(fri_layers[0]) # 8192 shoud be a prior knowledge
assert verify_decommitment(idx % length, v_cp0[0], v_cp0[1], cp_root.root)

50f954f4318abdc3779fb24e1eea92c6284aaf8978c453e21afe960b37a10698
50f954f4318abdc3779fb24e1eea92c6284aaf8978c453e21afe960b37a10698
6580468d0c0f1601e03acca5ea0ac3e746637455f0f85c58abb4ba65bbdf446e


In [52]:
# Check the polynomial constraint
fx = v_f_eval[0]
fgx = v_f_eval[1]
x = H_w[idx]
p0 = fx / (x - G[0])
p1 = (fx - FieldElement(99, Field.main()))/ (x - G[99])
p2_num = fgx - fx - FieldElement(1, Field.main())
p2_dem = FieldElement(1, Field.main())
for i in range(1, 99):
    p2_dem *= (x - G[i])

p2 = p2_num/p2_dem
assert v_cp0[0] == (p0 + p1 + p2)
print("poly constraint success!")

poly constraint success!


In [53]:
# Check low degree
v_cp = proof_cp[::2]
v_auth = proof_cp[1::2]
assert len(v_cp) == len(v_auth)
assert len(v_cp) % 2 == 0
k = length
for i in range(len(v_cp)//2 -1):
    iter_idx = idx % k
    iter_sib_idx = (idx + k // 2) % k
    assert verify_decommitment(iter_idx, v_cp[2*i], v_auth[2*i], fri_merkles[i].root)
    assert verify_decommitment(iter_sib_idx, v_cp[(2*i) + 1], v_auth[(2*i)+1], fri_merkles[i].root)
    k = k // 2

6580468d0c0f1601e03acca5ea0ac3e746637455f0f85c58abb4ba65bbdf446e
6580468d0c0f1601e03acca5ea0ac3e746637455f0f85c58abb4ba65bbdf446e
f7a3ad4bda829563a14197aba449586afeb35ca620265a47f19df9bb335eee61
f7a3ad4bda829563a14197aba449586afeb35ca620265a47f19df9bb335eee61
2cbfaa1a97184b654b4fb8425772fab04513fc36b323b0a3df6421676831d072
2cbfaa1a97184b654b4fb8425772fab04513fc36b323b0a3df6421676831d072
c77ec69880abc94f18d360f46e631ea6aa9bc6896e36edfafcdc645148af8200
c77ec69880abc94f18d360f46e631ea6aa9bc6896e36edfafcdc645148af8200
196456653b3c104012db22662ca5578aeba294ecfc999eb673324da56f702932
196456653b3c104012db22662ca5578aeba294ecfc999eb673324da56f702932
760565118e9b08133af9564a41190e8ca96420dc0823d50224c4ff534c9196ae
760565118e9b08133af9564a41190e8ca96420dc0823d50224c4ff534c9196ae


In [55]:
k = length
x = H_w[idx]
beta = [FieldElement(1, Field.main()) for i in range(99)]
for i in range(len(v_cp)//2 - 1):
    op1 = (v_cp[2*i] + v_cp[2*i + 1]) / FieldElement(2, Field.main())
    op2 = (v_cp[2*i] - v_cp[2*i + 1]) / (FieldElement(2, Field.main())*x)
    rhs = op1 + beta[i] * op2
    assert v_cp[2*(i+1)] == rhs, f" round {i}, CP(i+1) is {v_cp[2*(i+1)]} while rhs is {rhs}"
    x = x^2
print("low degree test success!")

low degree test success!
